In [1]:
import pandas as pd
import random
import kagglehub
import os
import json

In [2]:
from conv_generation_gpt_api import generate_natural_conversation_step1, generate_natural_conversation_step2, generate_natural_conversation_step3, generate_natural_conversation_step4

## 1. set the elements for conversation template

In [3]:
# 从名字脚本里面随机选择四个名字
# Download latest version
path = kagglehub.dataset_download("ryanburnsworth/popular-names-by-birth-year-1880-2022")
print("Path to dataset files:", path)

# List all files in the directory to find the correct CSV file
# files = os.listdir(path)
# print("Files in directory:", files)

# Load the CSV file into a pandas DataFrame
file_path = os.path.join(path, 'names_by_birth_year.csv')
df = pd.read_csv(file_path)
print(df.head())

Path to dataset files: /Users/liuyiwei/.cache/kagglehub/datasets/ryanburnsworth/popular-names-by-birth-year-1880-2022/versions/1
   Year       Name Gender  Count
0  1883       Mary      F   8012
1  1883       Anna      F   3306
2  1883       Emma      F   2367
3  1883  Elizabeth      F   2255
4  1883     Minnie      F   2035


In [4]:
# 1. 将所有离开原因写成一个列表
leave_reasons = [
    "bathroom break",
    "coffee break",
    "forgot something important",
    "forgot to print some documents",
    "forgot to recieve a package",
    "forgot to return a package",
    "forgot to run errands",
    "forgot to submit documents",
    "have a meeting starting soon that I need to prepare for",
    "have a previous engagement that I need to attend to quickly",
    "have a work-related emergency that requires my immediate attention",
    "have an unexpected visitor at my door",
    "have errands to run",
    "have to attend to someone who just walked in",
    "have to check on something",
    "have to go to the restroom",
    "have to pick up a prescription",
    "have to pick up dry cleaning",
    "have to print or scan documents",
    "have to receive a delivery",
    "have to recharge laptop",
    "have to return a borrowed item",
    "have to take care of a family matter",
    "have to take care of an unexpected task",
    "have unexpected visitor",
    "his/her pet needs attention",
    "his/her family is calling",
    "incoming delivery",
    "must respond to a phone call",
    "need to check on a friend or family member who needs assistance",
    "need to finish a task that’s time-sensitive",
    "need to get a phone call",
    "need to get some coffee",
    "need to go to the toilet",
    "need to grab a snack or a drink",
    "need to have a quick chat with someone else",
    "need to make a phone call",
    "need to make a quick trip to the drug store",
    "need to make a quick trip to the grocery store",
    "need to pick up a package",
    "need to receive a parcel",
    "need to recharge cellphone",
    "need to register for an event",
    "need to schedule a haircut or salon appointment",
    "need to schedule another appointment",
    "need to step away for a moment to stretch and clear my mind",
    "need to step out for a moment",
    "need to submit some papers",
    "need to take care of some paperwork or documents",
    "need to take care of some personal matters",
    "need to take care of something related to my health",
    "need to take care of something urgent",
    "need to troubleshoot something",
    "parking meter expiring",
    "remembered something that needs to be taken care of",
    "remembered to receive a package",
    "remembered to submit some papers",
    "remembered to take care of some paperwork or documents",
    "remembered to take care of some personal matters",
    "remembered to take care of something urgent",
    "want to go grab a drink",
    "want to go grab a coffee",
    "want to go take some fresh air",
    "want to go to the bathroom"
]

# # 3. 测试输出
# print("leave_reason_B:", leave_reason_B)
# print("leave_reason_D:", leave_reason_D)

In [5]:
# Function to populate template with variables
def populate_template(template, scenario, relationship, situation, lie_objective, real_reason, the_lie, the_fact, the_emotion_to_avoid, A_name, B_name, C_name, D_name):
    return template.replace("{{Topic for the scenario}}", scenario)\
                   .replace("{{relationship descriptor}}", relationship)\
                   .replace("{{A: the liar name}}", A_name)\
                   .replace("{{B: the target name}}", B_name)\
                   .replace("{{C: the accomplice name}}", C_name)\
                   .replace("{{D: the observer name}}", D_name)\
                   .replace("{{leave reason B}}", leave_reason_B)\
                   .replace("{{leave reason D_1}}", leave_reason_D_1)\
                   .replace("{{leave reason D_2}}", leave_reason_D_2)\
                   .replace("{{real_reason_c}}", real_reason_c)\
                   .replace("{{truth_c}}", truth_c if truth_c is not None else "")\
                   .replace("{{the lie}}", lie_c)\
                   .replace("{{situation_topic}}", situation_topic)\
                   .replace("{{situation}}", situation)\
                   .replace("{{lie_objective}}", lie_objective)

def replace_AB_with_name(text: str, A_name: str, B_name: str) -> str:
    """
    将文本中的 "A's" 替换为 "{A_name}'s"；"A" 替换为 "{A_name}"；
    同理 "B's" 替换为 "{B_name}'s"；"B" 替换为 "{B_name}"。
    """
    result = text.replace("A", f"{A_name}")
    result = result.replace("B", f"{B_name}")
    return result

In [38]:
# list for situation
# concert
# shopping
# painting
# ice skating

## 2. template

In [6]:
random_names = df['Name'].sample(n=4).to_list()
A_name, B_name, C_name, D_name = random_names
print(f"A_name = {A_name}")
print(f"B_name = {B_name}")
print(f"C_name = {C_name}")
print(f"D_name = {D_name}")

A_name = Tilmon
B_name = Frederic
C_name = Hurain
D_name = Weltha


In [7]:
leave_reason_B, leave_reason_D_1, leave_reason_D_2 = random.sample(leave_reasons, 3)
print(leave_reason_B)
print(leave_reason_D_1)
print(leave_reason_D_2)

want to go grab a drink
forgot something important
have to recharge laptop


### 2.0 import elements from the conv_element.json

In [13]:
import json

with open("/Users/liuyiwei/Desktop/TactfulToM/dataset/Tactful_conv_element.json", "r", encoding="utf-8") as f:
    data_list = json.load(f)

In [14]:
data = data_list[1]
print(data)
data

{'set_id': '2-0-1', 'lie_id': '0-1', 'conv_id': 2, 'truth_id': 1, 'lie_type': 'altruistic_white_lies', 'emotion': 'sad', 'topic': {'scenario': 'music taste', 'situation_topic': 'the plan for today after work', 'situation': 'Arnetta is inviting Marguerite to attend a old school jazz concert tonight', 'lie_objective': 'decline the invitation', 'leave_reason_B': 'have unexpected visitor', 'leave_reason_D_1': 'need to take care of some paperwork or documents', 'leave_reason_D_2': 'need to get a phone call'}, 'relationship': 'coworkers', 'characters': {'liar': 'Marguerite', 'target': 'Arnetta', 'accomplice': 'Antwann', 'observer': 'Jeanmichael'}, 'lie': {'real_reason_c': 'Marguerite is very particular about music and believes herself/himself has good taste. Marguerite dislikes Arnetta’s preferred style but avoids criticizing it in front of him/her.', 'lie_c': 'Marguerite has dinner researvation.', 'truth_c': 'Marguerite will go home after work.', 'falsification': True}, 'full_context': "**M

{'set_id': '2-0-1',
 'lie_id': '0-1',
 'conv_id': 2,
 'truth_id': 1,
 'lie_type': 'altruistic_white_lies',
 'emotion': 'sad',
 'topic': {'scenario': 'music taste',
  'situation_topic': 'the plan for today after work',
  'situation': 'Arnetta is inviting Marguerite to attend a old school jazz concert tonight',
  'lie_objective': 'decline the invitation',
  'leave_reason_B': 'have unexpected visitor',
  'leave_reason_D_1': 'need to take care of some paperwork or documents',
  'leave_reason_D_2': 'need to get a phone call'},
 'relationship': 'coworkers',
 'characters': {'liar': 'Marguerite',
  'target': 'Arnetta',
  'accomplice': 'Antwann',
  'observer': 'Jeanmichael'},
 'lie': {'real_reason_c': 'Marguerite is very particular about music and believes herself/himself has good taste. Marguerite dislikes Arnetta’s preferred style but avoids criticizing it in front of him/her.',
  'lie_c': 'Marguerite has dinner researvation.',
  'truth_c': 'Marguerite will go home after work.',
  'falsificat

In [15]:
# 最外层直接取值
set_id = data["set_id"]
lie_id = data["lie_id"]
conv_id = data["conv_id"]
truth_id = data["truth_id"]
lie_type = data["lie_type"]
emotion = data["emotion"]

# 取 topic 下面的嵌套字段
scenario = data["topic"]["scenario"]
situation_topic = data["topic"]["situation_topic"]
situation = data["topic"]["situation"]
lie_objective = data["topic"]["lie_objective"]
leave_reason_B = data["topic"]["leave_reason_B"]
leave_reason_D_1 = data["topic"]["leave_reason_D_1"]
leave_reason_D_2 = data["topic"]["leave_reason_D_2"]

relationship = data["relationship"]

# 取 characters 下面的嵌套字段
A_name = data["characters"]["liar"]
B_name = data["characters"]["target"]
C_name = data["characters"]["accomplice"]
D_name = data["characters"]["observer"]

# 取 lie 下面的嵌套字段
real_reason_c = data["lie"]["real_reason_c"]
lie_c = data["lie"]["lie_c"]
truth_c = data["lie"]["truth_c"]
falsification = data["lie"]["falsification"]

# 最外层的上下文
full_context = data["full_context"]
short_context = data["short_context"]


### 2.1 set the elements for prompt

In [ ]:
# Set your variables
set_id = "2-0-2"
lie_id = "0-2"
conv_id = 0
truth_id = 1

# falsification = None  # 赋值为空值（null）
# falsification = True
falsification = False

lie_type = "altruistic_white_lies"
emotion = "sad"

situation_topic = "the plan for today after work"
situation = "B is inviting A to attend a jazz obsession concert tonight"
lie_objective = "decline the invitation"

scenario = "music taste"
relationship = "coworkers"


real_reason_c = "A is very particular about music and believes herself/himself has good taste. A dislikes B’s preferred style but avoids criticizing it in front of him/her."
lie_c = "A already has some plans."
truth_c = "A will go home after work."

real_reason_c = replace_AB_with_name(real_reason_c, A_name, B_name)
lie_c = replace_AB_with_name(lie_c, A_name, B_name)
truth_c = replace_AB_with_name(truth_c, A_name, B_name)
situation = replace_AB_with_name(situation, A_name, B_name)

### 2.2 put elements into generation prompt

In [203]:
########## 提供truth!! ################
step_1 = """Start the conversation with all participants discussing {{Topic for the scenario}}.
In this conversation, four characters are participating: {{A: the liar name}}, {{B: the target name}}, {{C: the accomplice name}} and {{D: the observer name}}.
They are {{relationship descriptor}}.
"""

step_2 = """Now, {{B: the target name}} leaves the conversation because of the reason: {{leave reason B}}, while {{D: the observer name}} leaves the conversation because of the reason: {{leave reason D_1}}}.
With {{B: the target name}} and {{D: the observer name}} absent, {{A: the liar name}} and {{C: the accomplice name}} continue their conversation, shifting the topic to '{{real_reason_c}}'. Ensure the dialogue flows naturally without directly repeating these exact words.
Remember to indicate that {{B: the target name}} and {{D: the observer name}} are leaving the conversation.
"""

# 提供truth
step_3 = """Now, only {{D: the observer name}} returns and rejoins the conversation after after previously leaving due to: {{leave reason D_1}}. Note that {{B: the target name}} has not returned to the conversation yet.
The conversation continues and naturally shifts to {{situation_topic}}. Do NOT catch up or recap details from the previous conversation. 
And then '{{truth_c}}' is natrually revealed in the conversation. Ensure the dialogue flows naturally without directly repeating these exact words.
After engaging briefly , {{D: the observer name}} leaves the conversation again because of the reason: {{leave reason D_2}}.
IMPORTANT:
- Clearly indicate through dialogue only (e.g., "I'm back," or "I have to leave again now.") that {{D: the observer name}} first returns and later departs again.
- Do NOT use third-person narrative or stage directions.
- Avoid mentioning details from previous conversations.
"""

# In response, {{A: the liar name}} tells a white lie that '{{the lie}}', in order to '{{lie_objective}}'.
step_4 = """Now {{B: the target name}} returns to the conversation after leaving the conversation because of the reason: {{leave reason B}}. 
First have {{B: the target name}} explicitly indicate the return through dialogue. Do NOT catch up or recap details from the previous conversation. 
Then the conversation continues, where natrurally {{situation}}. In response, {{A: the liar name}} tells {{B: the target name}} that '{{the lie}}', in order to '{{lie_objective}}'. 
IMPORTANT:
- Do NOT use third-person narrative or stage directions.
- Avoid mentioning details from previous conversations.
"""

In [233]:
########## 不提供truth!! ################
step_1 = """Start the conversation with all participants discussing {{Topic for the scenario}}.
In this conversation, four characters are participating: {{A: the liar name}}, {{B: the target name}}, {{C: the accomplice name}} and {{D: the observer name}}.
They are {{relationship descriptor}}.
"""

step_2 = """Now, {{B: the target name}} leaves the conversation because of the reason: {{leave reason B}}, while {{D: the observer name}} leaves the conversation because of the reason: {{leave reason D_1}}}.
With {{B: the target name}} and {{D: the observer name}} absent, {{A: the liar name}} and {{C: the accomplice name}} continue their conversation, shifting the topic to '{{real_reason_c}}'. Ensure the dialogue flows naturally without directly repeating these exact words.
Remember to indicate that {{B: the target name}} and {{D: the observer name}} are leaving the conversation.
"""
# 不提供truth:
step_3 = """Now, only {{D: the observer name}} returns and rejoins the conversation after after previously leaving due to: {{leave reason D_1}}. Note that {{B: the target name}} has not returned to the conversation yet.
The conversation continues and naturally shifts to {{situation_topic}}. Do NOT catch up or recap details from the previous conversation.
After engaging briefly , {{D: the observer name}} leaves the conversation again because of the reason: {{leave reason D_2}}.
IMPORTANT:
- Clearly indicate through dialogue only (e.g., "I'm back," or "I have to leave again now.") that {{D: the observer name}} first returns and later departs again.
- Do NOT use third-person narrative or stage directions.
- Avoid mentioning details from previous conversations.
"""

step_4 = """Now {{B: the target name}} returns to the conversation after leaving the conversation because of the reason: {{leave reason B}}. 
First have {{B: the target name}} explicitly indicate the return through dialogue. Do NOT catch up or recap details from the previous conversation. 
Then the conversation continues, where natrurally {{situation}}. In response, {{A: the liar name}} tells {{B: the target name}} that '{{the lie}}', in order to '{{lie_objective}}'. 
IMPORTANT:
- Do NOT use third-person narrative or stage directions.
- Avoid mentioning details from previous conversations.
"""

In [234]:
# Populate template
filled_step_1 = populate_template(step_1, scenario, relationship, situation, lie_objective, real_reason_c, lie_c, truth_c, emotion, A_name, B_name, C_name, D_name)
filled_step_2 = populate_template(step_2, scenario, relationship, situation, lie_objective, real_reason_c, lie_c, truth_c, emotion, A_name, B_name, C_name, D_name)
filled_step_3 = populate_template(step_3, scenario, relationship, situation, lie_objective, real_reason_c, lie_c, truth_c, emotion, A_name, B_name, C_name, D_name)
filled_step_4 = populate_template(step_4, scenario, relationship, situation, lie_objective, real_reason_c, lie_c, truth_c, emotion, A_name, B_name, C_name, D_name)
print(filled_step_1)
print(filled_step_2)
print(filled_step_3)
print(filled_step_4)

Start the conversation with all participants discussing music taste.
In this conversation, four characters are participating: Marvis, Clair, Carolyn and Keyunta.
They are coworkers.

Now, Clair leaves the conversation because of the reason: need to get some coffee, while Keyunta leaves the conversation because of the reason: need to get a phone call}.
With Clair and Keyunta absent, Marvis and Carolyn continue their conversation, shifting the topic to 'Marvis takes great pride in their music taste and considers it superior to others, and dislike Clair’s preferred style.'. Ensure the dialogue flows naturally without directly repeating these exact words.
Remember to indicate that Clair and Keyunta are leaving the conversation.

Now, only Keyunta returns and rejoins the conversation after after previously leaving due to: need to get a phone call. Note that Clair has not returned to the conversation yet.
The conversation continues and naturally shifts to the life after work. Do NOT catch up

### 2.3 generation

### step 1

In [84]:
result_1 = generate_natural_conversation_step1(filled_step_1)

In [85]:
part_1 = result_1.choices[0].message['content']
print(part_1)

**Marvis:** So, what's everyone's go-to genre of music? I'm curious to know what gets you all pumped during work.

**Clair:** I tend to lean towards indie and alternative. There's something about those raw, unique sounds that just click with me. What about you, Carolyn?

**Carolyn:** I'm a bit of a jazz enthusiast. There's nothing like the smooth, complex beats to help me focus. Plus, it just feels timeless, you know? Keyunta, how about you?

**Keyunta:** I'm all about hip-hop. The rhythms and lyrics are like poetry to me, and it really motivates me during those long projects. How about you, Marvis?

**Marvis:** I'm actually into classic rock. I love the energy and the riffs. There's just something about it that keeps me going, especially when deadlines are approaching!

**Clair:** That's quite a mix of genres we have here! Have any of you tried expanding your playlists based on recommendations from friends or apps?

**Carolyn:** I have. Actually, I found some fantastic jazz fusion art

### step 2

In [151]:
input_2 = filled_step_2 + "\n"+ part_1
print(input_2)

Now, Clair leaves the conversation because of the reason: need to get some coffee, while Keyunta leaves the conversation because of the reason: need to get a phone call}.
With Clair and Keyunta absent, Marvis and Carolyn continue their conversation, shifting the topic to 'Marvis takes great pride in their music taste and considers it superior to others, and dislike Clair’s preferred style.'. Ensure the dialogue flows naturally without directly repeating these exact words.
Remember to indicate that Clair and Keyunta are leaving the conversation.

**Marvis:** So, what's everyone's go-to genre of music? I'm curious to know what gets you all pumped during work.

**Clair:** I tend to lean towards indie and alternative. There's something about those raw, unique sounds that just click with me. What about you, Carolyn?

**Carolyn:** I'm a bit of a jazz enthusiast. There's nothing like the smooth, complex beats to help me focus. Plus, it just feels timeless, you know? Keyunta, how about you?

*

In [153]:
result_2 = generate_natural_conversation_step2(input_2)
part_2 = result_2.choices[0].message['content']
print(part_2)

**Clair:** Awesome, but I need to grab some coffee first. I'll catch up with you all in a bit!

**Keyunta:** And I better take this phone call. I'll be back soon too!

**Marvis:** Now that it's just us, Carolyn, I've got to say, I just can't wrap my head around Clair's preference for indie and alternative. It feels so... unstructured compared to the classics.

**Carolyn:** I get where you’re coming from. Jazz sometimes gets misunderstood too. Everyone’s got their tastes. What’s it about classic rock that makes you so enthusiastic?

**Marvis:** It's the energy and the storytelling. Most of those songs are like mini-narratives, powerful and moving. Indie feels too abstract to me. Give me a good guitar riff any day.

**Carolyn:** That makes sense. It's the complexity in jazz that does it for me, though. I love how it keeps you guessing and evokes such emotion. You ever give jazz a try?

**Marvis:** A few times, mostly because you recommended them. It’s growing on me, though guitars will a

### step 3

In [226]:
input_3 = filled_step_3 + "\n" + part_1 + "\n" + part_2
print(input_3)

Now, only Keyunta returns and rejoins the conversation after after previously leaving due to: need to get a phone call. Note that Clair has not returned to the conversation yet.
The conversation continues and naturally shifts to the life after work. Do NOT catch up or recap details from the previous conversation.
After engaging briefly , Keyunta leaves the conversation again because of the reason: remembered to take care of some paperwork or documents.
IMPORTANT:
- Clearly indicate through dialogue only (e.g., "I'm back," or "I have to leave again now.") that Keyunta first returns and later departs again.
- Do NOT use third-person narrative or stage directions.
- Avoid mentioning details from previous conversations.

**Marvis:** So, what's everyone's go-to genre of music? I'm curious to know what gets you all pumped during work.

**Clair:** I tend to lean towards indie and alternative. There's something about those raw, unique sounds that just click with me. What about you, Carolyn?

*

In [228]:
result_3 = generate_natural_conversation_step3(input_3)
part_3 = result_3.choices[0].message['content']
print(part_3)

**Keyunta:** I'm back! Sorry about the delay. What'd I miss?

**Carolyn:** We were just chatting about our music tastes and thinking about starting a shared playlist with some of our favorites. How does that sound to you?

**Keyunta:** That sounds like an epic idea! I’d love to share some of my hip-hop favorites and discover what everyone else is into.

**Marvis:** Absolutely. It promises to be a great way to step out of our musical comfort zones and maybe catch some vibes for after-work downtime.

**Keyunta:** Definitely. Speaking of after work, do you guys have plans for winding down? I usually hit the gym or watch some documentaries.

**Carolyn:** I tend to relax with a book or sometimes a nice stroll around the park. It helps to decompress after a hectic day.

**Keyunta:** Both sound great. Just remembered I have to sort out some paperwork, so I better head out now. Duty calls! I'll catch up with you guys later.

**Marvis:** Take care, Keyunta! Don't let that paperwork pile up!


### step 4

In [237]:
input_4 = filled_step_4+ "\n" + part_1 + "\n" + part_2 + "\n" +part_3
print(input_4)

Now Clair returns to the conversation after leaving the conversation because of the reason: need to get some coffee. 
First have Clair explicitly indicate the return through dialogue. Do NOT catch up or recap details from the previous conversation. 
Then the conversation continues, where natrurally Clair is inviting Marvis to attend an concert tonight of her favourite band. In response, Marvis tells Clair that 'Marvis has a dinner with friends tonight.', in order to 'decline the invitation'. 

IMPORTANT:
- Do NOT use third-person narrative or stage directions.
- Avoid mentioning details from previous conversations.

**Marvis:** So, what's everyone's go-to genre of music? I'm curious to know what gets you all pumped during work.

**Clair:** I tend to lean towards indie and alternative. There's something about those raw, unique sounds that just click with me. What about you, Carolyn?

**Carolyn:** I'm a bit of a jazz enthusiast. There's nothing like the smooth, complex beats to help me f

In [238]:
result_4 = generate_natural_conversation_step4(input_4)
part_4 = result_4.choices[0].message['content']
print(part_4)

**Clair:** I'm back with my coffee! I hope I didn't miss too much of the conversation. How's everyone doing?

**Carolyn:** Welcome back, Clair! We were just talking about sharing our favorite tracks to make a group playlist. You in?

**Clair:** Absolutely, count me in! I'll make sure to add some of my indie favorites. Speaking of music, my favorite band is playing tonight. I'd love for you to join me, Marvis. What do you say?

**Marvis:** That sounds amazing, but I actually have a dinner planned with some friends tonight. Maybe next time?

**Clair:** No worries! Rain check then. We should all try to catch a concert together sometime. It’d be fun to experience music live with you all!

**Carolyn:** Definitely! That sounds like a plan. Live music always has a special energy, and it'd be great to enjoy it together.

**Marvis:** Yes, for sure. Let’s keep an eye on upcoming events. I’m sure there’s something that will catch all our interests.

**Clair:** Agreed. Until then, let's keep build

### combine the conversation

In [ ]:
import tiktoken
# 以 GPT-4 模型为例，选择对应的模型编码器
encoding = tiktoken.encoding_for_model("gpt-4o")
full_context_tokens = encoding.encode(full_context)
short_context_tokens = encoding.encode(short_context)
# 分别打印长度（字符数）和token数
print(f"Full context length (chars): {len(full_context)}")
print(f"Full context length (tokens): {len(full_context_tokens)}")

print(f"Short context length (chars): {len(short_context)}")
print(f"Short context length (tokens): {len(short_context_tokens)}")

Full context length (chars): 5430
Full context length (tokens): 1198
Short context length (chars): 3295
Short context length (tokens): 748


In [240]:
full_context = part_1 + part_2 + part_3 + part_4
full_context

"**Marvis:** So, what's everyone's go-to genre of music? I'm curious to know what gets you all pumped during work.\n\n**Clair:** I tend to lean towards indie and alternative. There's something about those raw, unique sounds that just click with me. What about you, Carolyn?\n\n**Carolyn:** I'm a bit of a jazz enthusiast. There's nothing like the smooth, complex beats to help me focus. Plus, it just feels timeless, you know? Keyunta, how about you?\n\n**Keyunta:** I'm all about hip-hop. The rhythms and lyrics are like poetry to me, and it really motivates me during those long projects. How about you, Marvis?\n\n**Marvis:** I'm actually into classic rock. I love the energy and the riffs. There's just something about it that keeps me going, especially when deadlines are approaching!\n\n**Clair:** That's quite a mix of genres we have here! Have any of you tried expanding your playlists based on recommendations from friends or apps?\n\n**Carolyn:** I have. Actually, I found some fantastic ja

In [241]:
short_context = part_2 + part_3 + part_4
short_context

"**Clair:** Awesome, but I need to grab some coffee first. I'll catch up with you all in a bit!\n\n**Keyunta:** And I better take this phone call. I'll be back soon too!\n\n**Marvis:** Now that it's just us, Carolyn, I've got to say, I just can't wrap my head around Clair's preference for indie and alternative. It feels so... unstructured compared to the classics.\n\n**Carolyn:** I get where you’re coming from. Jazz sometimes gets misunderstood too. Everyone’s got their tastes. What’s it about classic rock that makes you so enthusiastic?\n\n**Marvis:** It's the energy and the storytelling. Most of those songs are like mini-narratives, powerful and moving. Indie feels too abstract to me. Give me a good guitar riff any day.\n\n**Carolyn:** That makes sense. It's the complexity in jazz that does it for me, though. I love how it keeps you guessing and evokes such emotion. You ever give jazz a try?\n\n**Marvis:** A few times, mostly because you recommended them. It’s growing on me, though g

## 3. json

In [246]:
# 将所有字段分组放进一个字典
data_dict = {
    "set_id": set_id,
    "lie_id": lie_id,
    "conv_id": conv_id,
    "truth_id": truth_id,
    "lie_type": lie_type,
    "emotion": emotion,
    "topic": {
        "scenario": scenario,
        "situation_topic": situation_topic,
        "situation": situation,
        "lie_objective": lie_objective,
        "leave_reason_B": leave_reason_B,
        "leave_reason_D_1": leave_reason_D_1,
        "leave_reason_D_2": leave_reason_D_2,
    },
    "relationship": relationship,
    "characters": {
        "liar": A_name,
        "target": B_name,
        "accomplice": C_name,
        "observer": D_name
    },
    "lie": {
        "real_reason_c": real_reason_c,
        "lie_c": lie_c,
        "truth_c": truth_c,
        "falsification": falsification,
    },
    "full_context": full_context,
    "short_context": short_context
}

In [ ]:
#  给justification questions生成一下正确错误选项
justification_qas = [
        {
            "question": f"Why did {liar} tell {target} '{lie}'?",
            "question_type": "justification:truth",
            "correct_answer": f"To politely decline without criticizing Clair’s taste, so won't hurt Clair’s feelings." 
            "wrong_answer":
            Because Marvis dislikes Clair personally.
            Because Marvis genuinely has another commitment.
            Because Marvis does not enjoy Clair’s favorite music.
            Marvis dislikes Clair personally.
            Marvis wants Clair to think they have an active social life.
            Marvis is ashamed of their real music preferences.
            To avoid hurting Clair’s feelings by criticizing her music taste.
            Marvis genuinely has other plans

In [245]:
# 写一个判断如果 'falsification' = null不写入 'truth_c'
data_dict

NameError: name 'data_dict' is not defined

In [248]:
def append_data_to_json(data_dict, filename="/Users/liuyiwei/Desktop/whiteToM/dataset/Tactful_conv_element.json"):
    # 1. 先尝试读取已有的 JSON 文件，如果不存在则创建一个空列表
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as f:
            try:
                data_list = json.load(f)
                # 如果文件不是一个列表，可能需要根据实际情况做容错处理
                if not isinstance(data_list, list):
                    data_list = []
            except json.JSONDecodeError:
                # 如果文件不是合法的 JSON 格式，就初始化为空列表
                data_list = []
    else:
        data_list = []
    
    # 2. 向列表中追加新的字典
    data_list.append(data_dict)
    
    # 3. 将更新后的列表重新写回 JSON 文件
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

append_data_to_json(data_dict, "/Users/liuyiwei/Desktop/TactfulToM/dataset/Tactful_conv_element.json")

In [ ]:
# gpt summarize一下real_reason_c用于问题生成
real_reason_q = "Alice’s social battery is really running dry these days, and she thinks she needs more time to be alone."
lie_q = "Alice has a meeting in the afternoon, so she won’t be able to join for studying."
truth_q = "Alice is actually free for the whole afternoon."

In [ ]:
# Define your data as a dictionary
data = {
    "set_id": "0-0-0",
    "lie_id": "0-0",
    "conv_id": 0,
    "truth_id": 0,
    "falsification": None,
    "emotion": "sad",
    "situation": "B is inviting A to study together",
    "lie_objective": "decline an invitation",
    "scenarios": "school life",
    "relationship": "friends",
    "characters": {
        "liar": "Alice",
        "target": "David",
        "accomplice": "Sam",
        "observer": "Tim"
    },
    "real_reason_1": "My social battery is really running dry these days, I think I need more time to be alone.",
    "lie_1": "I have a meeting in the afternoon, so I won’t be able to join you for studying then.",
    "truth_1": "I am free for the whole afternoon",
    "real_reason": "Alice’s social battery is really running dry these days, and she thinks she needs more time to be alone.",
    "lie": "Alice has a meeting in the afternoon, so she won’t be able to join for studying.",
    "truth": "Alice is actually free for the whole afternoon."
    "full_context": " ",
    "short_context": " "
}